<a href="https://colab.research.google.com/github/jeduardosleite/sql_basic_7/blob/main/C%C3%B3pia_de_sql_mod7_exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving cliente.csv to cliente (1).csv
Saving extra.csv to extra.csv
Saving giraffas.csv to giraffas.csv
Saving magalu.csv to magalu.csv
Saving postoshell.csv to postoshell.csv
Saving query_1.csv to query_1 (1).csv
Saving query_2_1.csv to query_2_1 (1).csv
Saving query_2_2.csv to query_2_2 (1).csv
Saving query_3.csv to query_3 (1).csv
Saving query_4.csv to query_4 (1).csv
Saving seveneleven.csv to seveneleven.csv
Saving shopee.csv to shopee.csv
Saving subway.csv to subway.csv
Saving transacoes.csv to transacoes (1).csv


In [ ]:
cliente = pd.read_csv('cliente.csv')
extra = pd.read_csv('extra.csv')
transacoes = pd.read_csv('transacoes.csv')
giraffas = pd.read_csv('giraffas.csv')
magalu = pd.read_csv('magalu.csv')
postoshell = pd.read_csv('postoshell.csv')
shopee = pd.read_csv('shopee.csv')
subway = pd.read_csv('subway.csv')

query_1 = pd.read_csv('query_1.csv')
query_2_1 = pd.read_csv('query_2_1.csv')
query_2_2 = pd.read_csv('query_2_2.csv')
query_3 = pd.read_csv('query_3.csv')
query_4 = pd.read_csv('query_4.csv')

# **Module 21 - Introduction SQL**

## **Lesson 7**

In this module i used concepts of:
- **SUBQUERIES**
- **PARTITIONED**
- **VIEWS**

## **Creating tables**

transacoes
```sql
CREATE EXTERNAL TABLE transacoes(
    id_cliente BIGINT,
    id_transacao BIGINT,
    data_compra DATE,
    valor DOUBLE,
    id_loja VARCHAR(25)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://mod7-jeduardo/transacao/';
```
<br>

cliente
```sql
CREATE EXTERNAL TABLE cliente(
    id_cliente BIGINT,
    nome VARCHAR(25),
    data_compra DATE,
    valor_compra FLOAT,
    loja_cadastrada VARCHAR(25)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://mod7-jeduardo/cliente/';
```

In [ ]:
cliente

,5,jose,2020-07-01,500.43,magalu
0,1,maria,2019-03-02,150.70,subway
1,2,valentina,2020-01-09,210.99,postoshell
2,4,joana,2019-05-11,1300.50,magalu
3,6,fernando,2020-03-02,86.55,seveneleven


In [ ]:
transacoes

,1,768805383,2021-06-10,50.74,magalu
0,2,768805399,2021-06-13,30.90,giraffas
1,3,818770008,2021-06-05,110.00,postoshell
2,1,76856563,2021-07-10,2000.90,magalu
3,1,767573759,2021-06-20,15.70,subway
4,3,818575758,2021-06-25,2.99,seveneleven
5,4,764545534,2021-07-10,50.74,extra
6,5,76766789,2021-08-22,10.00,subway
7,3,8154567758,2021-08-15,1100.00,shopee


### Query 1

```sql
SELECT id_loja, id_cliente, id_transacao
FROM transacoes
WHERE id_loja IN
  (SELECT loja_cadastrada
   FROM cliente
   WHERE valor_compra > 160);
```


In [ ]:
query_1

,id_loja,id_cliente,id_transacao
0,magalu,1,768805383
1,postoshell,3,818770008
2,magalu,1,76856563


### Query 2

#### **Creating partitioned**

Firstly, i created seven bucket with csv archives:
- id_loja = magalu
- id_loja = giraffas
- id_loja = postosshell
- id_loja = subway
- id_loja = seveneleven
- id_loja = extra
- id_loja = shopee

Now, i created a new table
```sql
CREATE EXTERNAL TABLE transacoes_part(
    id_cliente BIGINT,
    id_transacoes BIGINT,
    valor DOUBLE)
    PARTITIONED BY (id_loja STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://transacoes-partition-jeduardo/';
```

After that, I taught Athena.

```sql
MSCK REPAIR TABLE transacoes_part;
```
---
#### **Checking**
To check if it worked, I carried out the test with the file ***query_2_2***.


***query_2_2***

Same output for them.

```sql
SELECT *
FROM transacoes_part
WHERE id_loja = 'magalu'
```

```sql
SELECT *
FROM transacoes
WHERE id_loja = 'magalu'
```

In [ ]:
query_2_2

,id_cliente,id_transacao,data_compra,valor,id_loja
0,1,768805383,2021-06-10,50.74,magalu
1,1,76856563,2021-07-10,2000.90,magalu


### **Query 3 - View**

Creating a new table.
```sql
CREATE VIEW transacoesv100 AS
SELECT
  id_cliente AS cliente,
  valor AS valor_compra,
  id_loja AS nome_loja
FROM transacoes
WHERE valor > 100;
```

The output from this view.
```sql
SELECT *
FROM transacoesv100
```

In [ ]:
query_3

,cliente,valor_compra,nome_loja
0,3,110.0,postoshell
1,1,2000.9,magalu
2,3,1100.0,shopee


### **Query 4 - View

Creating a new table.
```sql
CREATE VIEW clientevalor AS
SELECT
  id_cliente AS cliente,
  valor AS valor_compra
FROM transacoes
ORDER BY valor DESC
LIMIT 2;
```

The output from this view:
```sql
SELECT *
FROM clientevalor
```

In [ ]:
query_4

,cliente,valor_compra
0,1,2000.9
1,3,1100.0


Speaking about views, we can use another functions as:
- ***DESCRIBE***
```sql
DESCRIBE clientevalor
```

- ***SHOW COLUMNS***
```sql
SHOW COLUMNS IN clientevalor
```

- ***SHOW VIEWS***
```sql
SHOW VIEWS
```

- ***SHOW CREATE VIEW***
```sql
SHOW CREATE VIEW clientevalor
```

- ***DROP VIEW***
```sql
DROP VIEW clientevalor
```